<a href="https://colab.research.google.com/github/Harsh7wardhan/Machine-Learning/blob/main/Word_embedding_enterpret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [ ]:
new_train=train_data[train_data['label']!=2]

In [ ]:
new_train.shape

(2974, 3)

In [ ]:
X=new_train.iloc[:,:-1]
y=new_train.iloc[:,-1]
print(X.shape)
print(y.shape)

(4000, 2)
(4000,)


In [ ]:
print(X)


                                                   text                              aspect
0       can you check whether its cancelled completely?                           cancelled
1     cannot rely on both milk delivery and grocery ...                                Milk
2     I get no notification, however the app is real...                        notification
3     Love this app, but would love it even more if ...                                view
4           it does not let me load a clip on the scene                                load
...                                                 ...                                 ...
3995  every time i try to edit a page or create a li...                               tools
3996  unable to load money using wallets (phonepe/ol...  Unable to load money using wallets
3997  hi, i m doing for the first time ever and i no...                               usage
3998  delivery is delayed or cancelled every time i ...                         

In [ ]:
print(y)


0       1
1       0
2       0
3       1
4       0
       ..
3995    0
3996    0
3997    0
3998    0
3999    0
Name: label, Length: 4000, dtype: int64


In [ ]:
y.value_counts()

0    1680
1    1294
2    1026
Name: label, dtype: int64

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__

'2.7.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()



In [ ]:
messages.reset_index(inplace=True)
messages['text']

0         can you check whether its cancelled completely?
1       cannot rely on both milk delivery and grocery ...
2       I get no notification, however the app is real...
3       Love this app, but would love it even more if ...
4             it does not let me load a clip on the scene
                              ...                        
3995    every time i try to edit a page or create a li...
3996    unable to load money using wallets (phonepe/ol...
3997    hi, i m doing for the first time ever and i no...
3998    delivery is delayed or cancelled every time i ...
3999                   your customer service is terrible!
Name: text, Length: 4000, dtype: object

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
print(corpus)
print(len(corpus))

['check whether cancel complet', 'cannot reli milk deliveri groceri item', 'get notif howev app realli fine', 'love app would love even gantt chart calendar view avail iphon', 'let load clip scene', 'unlimit yearli year', 'time miss deliv time differ order deliv', 'would love abil un delet delet task app easi find menu option', 'want download app', 'gt want learn futur whether team someth wrong whether teammat accid delet file gt therefor would good get mailadress kaiser got hack whether failur notion system', 'app must support android multi window api would better tablet resiz app screen', 'ensur custom get deliveri everyday inform advanc case deliveri failur', 'go export tab see export video', 'use istock media month', 'use app min io crash date select imposs option seemingli use white font white background app look good dark mode reason white white ui issu', 'thought link databas favourit featur notionhq', 'differ regular milkman infact charg less deliveri charg', 'item miss logic l

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  117 3348 1010]
 [   0    0    0 ...  832 3204 3421]
 [   0    0    0 ... 1653 4034 4395]
 ...
 [   0    0    0 ... 2104 1739 1653]
 [   0    0    0 ... 1839 4029 2481]
 [   0    0    0 ... 2610  117 2239]]


In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs)

2974

In [ ]:
y.shape

(2974,)

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((2974, 20), (2974,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=0)

Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
47/47 [==============================] - 4s 25ms/step - loss: 0.4466 - accuracy: 0.3197 - val_loss: 0.4619 - val_accuracy: 0.3330
Epoch 2/10
47/47 [==============================] - 1s 12ms/step - loss: 0.2190 - accuracy: 0.3290 - val_loss: 0.0944 - val_accuracy: 0.3600
Epoch 3/10
47/47 [==============================] - 1s 12ms/step - loss: -0.9215 - accuracy: 0.4223 - val_loss: -0.3921 - val_accuracy: 0.3950
Epoch 4/10
47/47 [==============================] - 1s 13ms/step - loss: -2.5353 - accuracy: 0.4880 - val_loss: -1.2842 - val_accuracy: 0.4600
Epoch 5/10
47/47 [==============================] - 1s 12ms/step - loss: -4.3050 - accuracy: 0.5203 - val_loss: -2.2596 - val_accuracy: 0.4400
Epoch 6/10
47/47 [==============================] - 1s 12ms/step - loss: -6.1546 - accuracy: 0.5307 - val_loss: -2.5608 - val_accuracy: 0.4670
Epoch 7/10
47/47 [==============================] - 1s 12ms/step - loss: -6.3583 - accuracy: 0.5453 - val_loss: -2.4834 - val_accuracy: 0.4530
Epo

In [ ]:
y_pred=model.predict(X_test).argmax(axis=1)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[435,   0,   0],
       [333,   0,   0],
       [232,   0,   0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)*100

43.5